# Maps of the search and homing path for light and dark trials



In [1]:
%load_ext autoreload
%autoreload 2

%run ~/repo/autopi_analysis_bk/Jazi_et.al_2023_noInt/setup_project.py
%run ~/repo/autopi_analysis_bk/Jazi_et.al_2023_noInt/neuronAutopi.py


prepareSessionsForSpatialAnalysisProject(sSesList,myProject.sessionList,pose_file_extension = ".pose_kf.npy" )

Project name: autopi_ca1
dataPath: /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1
dlcModelPath: /adata/models
Reading /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/sessionList
We have 39 testing sessions in the list
See myProject and sSesList objects
Loading Animal_pose and Spike_train, sSes.ap and sSes.cg


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:08<00:00,  4.36it/s]

Loading ses.trial_table_simple as ses.trials
Create condition intervals in ses.intervalDict


Get intervals for the search and homing paths

In [2]:
for ses, sSes in tqdm(zip(myProject.sessionList,sSesList)):
    getSearchHomingIntervals(ses,sSes)

39it [00:00, 48.37it/s]


In [3]:
def getMaps(ses,sSes,onlyArena=False):
    """
    Get all the maps of the data within the intervals of interest
    """
    
    
    xy_range=np.array([[-50,-90],[50,60]])
    
    # get all the conditions we are interested
    conditionDicts = [{"type" : "searchToLeverPath", "light" : "light"},
                      {"type" : "searchToLeverPath", "light" : "light_1"},
                      {"type" : "searchToLeverPath", "light" : "light_2"},
                      {"type" : "searchToLeverPath", "light" : "dark"},
                      {"type" : "searchToLeverPath", "light" : "dark_1"},
                      {"type" : "searchToLeverPath", "light" : "dark_2"},
                      {"type" : "homingFromLeavingLever", "light" : "light"},
                      {"type" : "homingFromLeavingLever", "light" : "light_1"},
                      {"type" : "homingFromLeavingLever", "light" : "light_2"},
                      {"type" : "homingFromLeavingLever", "light" : "dark"},
                      {"type" : "homingFromLeavingLever", "light" : "dark_1"},
                      {"type" : "homingFromLeavingLever", "light" : "dark_2"}]

    res = {}
    for d in conditionDicts:
    
        navPathType= d["type"]
        light = d["light"]
        inter = ses.intervalDict[navPathType+"_"+light]

        sSes.ap.set_intervals(inter)
        
        if onlyArena:
            # this should come after setting the time intervals, will be reset when calling sSes.ap.set_intervals again
            sSes.ap.invalid_outside_spatial_area(shape="circle",radius=43.0,center=np.array([0,0])) ## remove the bridge and homebase

        for n in sSes.cg.neuron_list:
            n.spike_train.set_intervals(inter)
            n.spatial_properties.firing_rate_map_2d(cm_per_bin =3, smoothing_sigma_cm = 5, smoothing=True,xy_range=xy_range)
    
        mapList = [ n.spatial_properties.firing_rate_map for n in sSes.cg.neuron_list ]
        mapStack = np.stack(mapList, axis=0)
        res[navPathType+"_"+light]=mapStack
        
    # reset to original intervals
    sSes.ap.unset_intervals()
        
    return res

## Try out on one session

In [4]:
ses = myProject.sessionList[1]
sSes = sSesList[1]

In [5]:
res = getMaps(ses,sSes)

## Run on all sessions

In [6]:
res = [ getMaps(ses,sSes) for ses, sSes in tqdm(zip(myProject.sessionList,sSesList))]

39it [05:15,  8.09s/it]


Restructure the results to get one stack of maps per condition.

In [7]:
newRes = {}
for k in res[0].keys():
    newRes[k] = np.concatenate([d[k] for d in res])

In [8]:
for k in newRes.keys():
    print(k,newRes[k].shape)

searchToLeverPath_light (1080, 34, 50)
searchToLeverPath_light_1 (1080, 34, 50)
searchToLeverPath_light_2 (1080, 34, 50)
searchToLeverPath_dark (1080, 34, 50)
searchToLeverPath_dark_1 (1080, 34, 50)
searchToLeverPath_dark_2 (1080, 34, 50)
homingFromLeavingLever_light (1080, 34, 50)
homingFromLeavingLever_light_1 (1080, 34, 50)
homingFromLeavingLever_light_2 (1080, 34, 50)
homingFromLeavingLever_dark (1080, 34, 50)
homingFromLeavingLever_dark_1 (1080, 34, 50)
homingFromLeavingLever_dark_2 (1080, 34, 50)


In [9]:
fn=myProject.dataPath+"/results/mySearchHomingMaps.pickle"
print("Saving:",fn)
with open(fn, 'wb') as handle:
    pickle.dump(newRes, handle)

Saving: /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/results/mySearchHomingMaps.pickle


# Maps in the zone of the arena that is between the bridge and the center of the arena

This is an analysis that I came up with to answer a comment raised by a reviewer. We focus on a region of the arena that is well covered in all 4 behavioral conditions.

In [10]:
def getMapsInZone(ses,sSes,cm_per_bin = 2.5):
    """
    Get all the maps of the data within the intervals of interest
    """
    
    xy_range=np.array([[-10,-40],[10,0]])
    
    # get all the conditions we are interested
    conditionDicts = [{"type" : "searchToLeverPath", "light" : "light"},
                      {"type" : "searchToLeverPath", "light" : "light_1"},
                      {"type" : "searchToLeverPath", "light" : "light_2"},
                      {"type" : "searchToLeverPath", "light" : "dark"},
                      {"type" : "searchToLeverPath", "light" : "dark_1"},
                      {"type" : "searchToLeverPath", "light" : "dark_2"},
                      {"type" : "homingFromLeavingLever", "light" : "light"},
                      {"type" : "homingFromLeavingLever", "light" : "light_1"},
                      {"type" : "homingFromLeavingLever", "light" : "light_2"},
                      {"type" : "homingFromLeavingLever", "light" : "dark"},
                      {"type" : "homingFromLeavingLever", "light" : "dark_1"},
                      {"type" : "homingFromLeavingLever", "light" : "dark_2"}]

    res = {}
    for d in conditionDicts:
    
        navPathType= d["type"]
        light = d["light"]
        inter = ses.intervalDict[navPathType+"_"+light]

        sSes.ap.set_intervals(inter)
        
        
        # limit the analysis to the zone of interest
        center = [0,-20]
        length = [20,40]
        sSes.ap.invalid_outside_spatial_area(environment_shape="rectangle",center=center,length=length)
        
        
        for n in sSes.cg.neuron_list:
            n.spike_train.set_intervals(inter)
            n.spatial_properties.firing_rate_map_2d(cm_per_bin =cm_per_bin, smoothing_sigma_cm = 5, smoothing=True,xy_range=xy_range)
    
        mapList = [ n.spatial_properties.firing_rate_map for n in sSes.cg.neuron_list ]
        mapStack = np.stack(mapList, axis=0)
        res[navPathType+"_"+light]=mapStack
        
    # reset to original intervals
    sSes.ap.unset_intervals()
        
    return res

In [11]:
ses = myProject.sessionList[1]
sSes = sSesList[1]

In [12]:
res = getMapsInZone(ses,sSes)

In [13]:
res = [ getMapsInZone(ses,sSes,cm_per_bin=2.5) for ses, sSes in tqdm(zip(myProject.sessionList,sSesList))]

39it [04:52,  7.50s/it]


In [14]:
newRes = {}
for k in res[0].keys():
    newRes[k] = np.concatenate([d[k] for d in res])

In [15]:
for k in newRes.keys():
    print(k,newRes[k].shape)

searchToLeverPath_light (1080, 8, 16)
searchToLeverPath_light_1 (1080, 8, 16)
searchToLeverPath_light_2 (1080, 8, 16)
searchToLeverPath_dark (1080, 8, 16)
searchToLeverPath_dark_1 (1080, 8, 16)
searchToLeverPath_dark_2 (1080, 8, 16)
homingFromLeavingLever_light (1080, 8, 16)
homingFromLeavingLever_light_1 (1080, 8, 16)
homingFromLeavingLever_light_2 (1080, 8, 16)
homingFromLeavingLever_dark (1080, 8, 16)
homingFromLeavingLever_dark_1 (1080, 8, 16)
homingFromLeavingLever_dark_2 (1080, 8, 16)


In [16]:
fn=myProject.dataPath+"/results/mySearchHomingMapsInZone.pickle"
print("Saving:",fn)
with open(fn, 'wb') as handle:
    pickle.dump(newRes, handle)

Saving: /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/results/mySearchHomingMapsInZone.pickle


Repeat with larger bins for population vector analysis.

In [17]:
res = [ getMapsInZone(ses,sSes,cm_per_bin=5.0) for ses, sSes in tqdm(zip(myProject.sessionList,sSesList))]
newRes = {}
for k in res[0].keys():
    newRes[k] = np.concatenate([d[k] for d in res])
for k in newRes.keys():
    print(k,newRes[k].shape)
fn=myProject.dataPath+"/results/mySearchHomingMapsInZone_5cm_per_bin.pickle"
print("Saving:",fn)
with open(fn, 'wb') as handle:
    pickle.dump(newRes, handle) 

39it [04:51,  7.47s/it]

searchToLeverPath_light (1080, 4, 8)
searchToLeverPath_light_1 (1080, 4, 8)
searchToLeverPath_light_2 (1080, 4, 8)
searchToLeverPath_dark (1080, 4, 8)
searchToLeverPath_dark_1 (1080, 4, 8)
searchToLeverPath_dark_2 (1080, 4, 8)
homingFromLeavingLever_light (1080, 4, 8)
homingFromLeavingLever_light_1 (1080, 4, 8)
homingFromLeavingLever_light_2 (1080, 4, 8)
homingFromLeavingLever_dark (1080, 4, 8)
homingFromLeavingLever_dark_1 (1080, 4, 8)
homingFromLeavingLever_dark_2 (1080, 4, 8)
Saving: /ext_drives/d80/Jazi_etal_2023_noInter/autopi_ca1/results/mySearchHomingMapsInZone_5cm_per_bin.pickle


In [18]:
print('we are done here!')

we are done here!
